In [1]:
import os
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.path as mplPath
import shapefile
from netCDF4 import Dataset
import dateutil.parser
from shapely.geometry import Polygon, Point

In [2]:
shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/SalishSea_July172019_2/SalishSea_July172019.shp"

In [3]:
data_df = gpd.read_file(shapefile_name)

In [4]:
numLayers = 5;
numSites = data_df.shape[0]
numTargetSites = numSites

outputDT = 60*60 

stepsPerDay = int(86400.0/ outputDT);
numStepsPerDT = stepsPerDay;
numStepsPerDT = int(outputDT/3600.0)

In [5]:
debug = False

In [6]:
# Define paths
paths = {
    'out': './results',
}


In [7]:
inputFileName = os.path.join(paths['out'], 'Oil_disperse_2019_5b_Turn_Point_Diluted_bitumen.nc')

pfile = xr.open_dataset(str(inputFileName), decode_cf=True)

In [8]:
lon = np.ma.filled(pfile.variables['lon'], np.nan)
lat = np.ma.filled(pfile.variables['lat'], np.nan)
time = np.ma.filled(pfile.variables['time'], np.nan)
z = np.ma.filled(pfile.variables['z'], np.nan)
probs = np.ma.filled(pfile.variables['decay_value'], np.nan)

In [9]:
numParticles = lon.shape[0]

trackDates = [];

for i in range(0,numParticles):
    print(time[i][0])
    #trackDates.append( dateutil.parser.parse(time[i][0]))
    trackDates.append(time[i][0])

2019-01-04T14:00:00.000000000
2019-01-04T21:00:00.000000000
2019-01-04T18:00:00.000000000
2019-01-04T10:00:00.000000000
2019-01-04T11:00:00.000000000
2019-01-04T00:00:00.000000000
2019-01-04T02:00:00.000000000
2019-01-04T15:00:00.000000000
2019-01-04T04:00:00.000000000
2019-01-04T03:00:00.000000000
2019-01-04T12:00:00.000000000
2019-01-04T16:00:00.000000000
2019-01-04T17:00:00.000000000
2019-01-04T21:00:00.000000000
2019-01-04T06:00:00.000000000
2019-01-04T12:00:00.000000000
2019-01-04T19:00:00.000000000
2019-01-04T16:00:00.000000000
2019-01-04T22:00:00.000000000
2019-01-04T05:00:00.000000000
2019-01-04T01:00:00.000000000
2019-01-04T00:00:00.000000000
2019-01-04T01:00:00.000000000
2019-01-04T00:00:00.000000000
2019-01-04T13:00:00.000000000
2019-01-04T13:00:00.000000000
2019-01-04T13:00:00.000000000
2019-01-04T12:00:00.000000000
2019-01-04T02:00:00.000000000
2019-01-04T22:00:00.000000000
2019-01-04T19:00:00.000000000
2019-01-04T22:00:00.000000000
2019-01-04T03:00:00.000000000
2019-01-04

In [10]:
RDiff = max(trackDates) - min(trackDates)

minDate = np.datetime64("2019-01-04T00:30:00");
ts = pd.to_datetime(str(minDate))
d = ts.strftime('%Y-%m-%d %H:%M:%S')
print(d)

2019-01-04 00:30:00


In [11]:
numReleaseDays = 1;
numReleaseSteps = numReleaseDays * stepsPerDay;
trackLength = len(lon[0]);

print('trackLength = ' + str(trackLength))
print('numStepsPerDT = ' + str(numStepsPerDT))

numSteps = int(trackLength / numStepsPerDT)

trackLength = 145
numStepsPerDT = 1


In [12]:
# Create the netcdf output file

netcdfFileName = "Atlantis_Oil_2019" + "_dt_" + str(outputDT) + ".nc"

try:
    os.remove(netcdfFileName)
except:
    pass

ncfile = Dataset(netcdfFileName, "w", format="NETCDF4", clobber=True)

# Dimensions
time = ncfile.createDimension("t", None)
b = ncfile.createDimension("b", numTargetSites)
z = ncfile.createDimension("z", numLayers)

In [13]:
# Variables
times = ncfile.createVariable("time","f4",("t",))
oil = ncfile.createVariable("oil","f4",("t", "b",))
Naphthalene = ncfile.createVariable("Naphthalene","f4",("t", "b",))
Phenanthrene = ncfile.createVariable("Phenanthrene","f4",("t", "b",))
Pyrene = ncfile.createVariable("Pyrene","f4",("t", "b",))
Benzo = ncfile.createVariable("Benzo","f4",("t", "b",))

# Attributes
times.units = "seconds since " + d
times.dt = str(outputDT);
oil.units = "mgPAH/m^3"

Naphthalene.units = "mgPAH/m^3"
Naphthalene.long_name = "Amount of Naphthalene"

Phenanthrene.units = "mgPAH/m^3"
Phenanthrene.long_name = "Amount of Phenanthrene"

Pyrene.units = "mgPAH/m^3"
Pyrene.long_name = "Amount of Pyrene"

Benzo.units = "mgPAH/m^3"
Benzo.long_name = "Amount of Benzo(a)pyrene"

# Populate variables with data
timeData = np.arange(0,(numSteps + numReleaseSteps)*outputDT,outputDT)
times[:] = timeData;

boxDispersal = np.zeros((numSteps + numReleaseSteps, numTargetSites));

In [14]:
for partIndex in range(0, numParticles):

    trackDateDiff = trackDates[partIndex] - minDate;
    trackDateDiff = trackDateDiff/ np.timedelta64(1, 's')

    timeOffset = int(abs((trackDateDiff /outputDT)));

    for stepIndex in range(0, numSteps):
        timeValue = stepIndex + timeOffset

        partLon = lon[partIndex][stepIndex * numStepsPerDT];
        partLat = lat[partIndex][stepIndex * numStepsPerDT];
        partProb = probs[partIndex][stepIndex * numStepsPerDT];

        matchFound = 0;

        for targetIndex in range(0, numTargetSites):

            path = data_df.iloc[targetIndex].geometry
            checks = path.contains(Point(partLon, partLat));

            if checks:
                boxDispersal[timeValue][targetIndex] = boxDispersal[timeValue][targetIndex] + partProb;

                # uncomment line below to ignore particle decay during debugging.
                #boxDispersal[timeValue][targetIndex] = boxDispersal[timeValue][targetIndex] + 1.0

                matchFound = 1
                if debug:
                    print('At time ' + str(timeValue) + ' Particle (' + str(partIndex) + ') in box ' + str(data_df.iloc[targetIndex].BOX_ID))


                break;

        if matchFound == 0:
            if debug:
                print('No match for particle')
                print(partLon, partLat)


        #break

oil[:, :] = boxDispersal;
Naphthalene[:, :] = boxDispersal;
Phenanthrene[:, :] = boxDispersal;
Pyrene[:, :] = boxDispersal;
Benzo[:, :] = boxDispersal;

ncfile.close()